## COde

In [22]:
playerA_json = {
  "meta": {
    "player_id": 14,
    "series_id": 2819695,
    "rounds_played": 2,
    "rounds_won": 0,
    "rounds_lost": 2
  },
  "series_strengths": [
    {
      "feature": "damage_efficiency",
      "mean_shap": 0.6944999694824219,
      "direction": "positive",
      "impact_level": "high",
      "consistency": 5.0995001792907715
    },
    {
      "feature": "shotgun_damage_ratio",
      "mean_shap": 0.09600000083446503,
      "direction": "positive",
      "impact_level": "low",
      "consistency": 0.006999999284744263
    }
  ],
  "series_weaknesses": [
    {
      "feature": "body_damage_ratio",
      "mean_shap": -0.16300000250339508,
      "direction": "negative",
      "impact_level": "moderate",
      "consistency": 0.012999996542930603
    },
    {
      "feature": "alive",
      "mean_shap": -0.06600000150501728,
      "direction": "negative",
      "impact_level": "low",
      "consistency": 0.01300000213086605
    },
    {
      "feature": "headshot_damage_ratio",
      "mean_shap": -0.057999998331069946,
      "direction": "negative",
      "impact_level": "low",
      "consistency": 0.0
    }
  ],
  "win_probability": {
    "mean": 0.12875000201165676,
    "median": 0.12875000201165676,
    "min": 0.04859999939799309,
    "max": 0.20890000462532043,
    "stability": 0.9198499973863363
  },
  "economy_profile": {
    "player_loadout_ratio": 0.192,
    "player_networth_ratio": 0.194,
    "money_left_ratio": 0.403
  },
  "eco_summary": {
    "summary": "Player is an economy-balanced contributor, contributing 19% of team firepower. They conserves credits well.",
    "labels": {
      "investment_style": "economy-balanced contributor",
      "credit_discipline": "conserves credits well"
    }
  },
  "weapon_profile": {
    "most_common_weapon": "rifle",
    "weapon_damage_ratio": {
      "rifle": 0.448,
      "smg": 0.0,
      "shotgun": 0.0,
      "sniper": 0.247,
      "pistol": 0.084,
      "ability": 0.221
    },
    "weapon_usage_ratio": {
      "rifle": 0.548,
      "smg": 0.018,
      "shotgun": 0.011,
      "sniper": 0.134,
      "pistol": 0.066,
      "ability": 0.223
    },
    "summary": "Primary rifle relying heavily on rifles"
  },
  "playstyle": "aim_heavy",
  "payer_consistency_score": 0.6700167504187605
}


playerB_json = {
  "eco_summary": {
    "labels": {
      "credit_discipline": "conserves credits well",
      "investment_style": "low-investment contributor"
    },
    "summary": "Player is an low-investment contributor, contributing 5% of team firepower. They conserves credits well."
  },
  "economy_profile": {
    "money_left_ratio": 0.4123333333333333,
    "player_loadout_ratio": 0.05833333333333333,
    "player_networth_ratio": 0.10899999999999999
  },
  "meta": {
    "player_id": "10612",
    "rounds_lost": 3,
    "rounds_played": 3,
    "rounds_won": 0,
    "series_id": "2843069"
  },
  "payer_consistency_score": 0.999058078965347,
  "playstyle": "aim_heavy",
  "series_strengths": [
    {
      "consistency": 0.0,
      "direction": "positive",
      "feature": "pistol_damage_ratio",
      "impact_level": "low",
      "mean_shap": 0.07900000363588333
    },
    {
      "consistency": 0.07699999772012234,
      "direction": "positive",
      "feature": "headshot_damage_ratio",
      "impact_level": "low",
      "mean_shap": 0.016999999061226845
    }
  ],
  "series_weaknesses": [
    {
      "consistency": 0.0,
      "direction": "negative",
      "feature": "damage_efficiency",
      "impact_level": "high",
      "mean_shap": -4.40500020980835
    },
    {
      "consistency": 0.03227314281511221,
      "direction": "negative",
      "feature": "alive",
      "impact_level": "low",
      "mean_shap": -0.08433333535989125
    },
    {
      "consistency": 0.0,
      "direction": "negative",
      "feature": "leg_damage_ratio",
      "impact_level": "low",
      "mean_shap": -0.05400000140070915
    },
    {
      "consistency": 0.1275521709743504,
      "direction": "negative",
      "feature": "body_damage_ratio",
      "impact_level": "low",
      "mean_shap": -0.04433333377043406
    }
  ],
  "weapon_profile": {
    "most_common_weapon": "rifle",
    "summary": "Primary rifle relying heavily on rifles",
    "weapon_damage_ratio": {
      "ability": 0.0,
      "pistol": 0.08,
      "rifle": 0.698,
      "shotgun": 0.0,
      "smg": 0.043,
      "sniper": 0.0
    },
    "weapon_usage_ratio": {
      "ability": 0.0,
      "pistol": 0.138,
      "rifle": 0.737,
      "shotgun": 0.0,
      "smg": 0.125,
      "sniper": 0.0
    }
  },
  "win_probability": {
    "max": 0.7501999735832214,
    "mean": 0.275299991791447,
    "median": 0.057500001043081284,
    "min": 0.018200000748038292,
    "stability": 0.6638119404904765
  }
}


In [23]:
import copy
import math
import numpy as np

# ---------- utilities ----------
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def logit(p, eps=1e-9):
    p = min(max(p, eps), 1 - eps)
    return math.log(p / (1 - p))

def clamp(x, lo=0.0, hi=1.0):
    return max(lo, min(hi, x))

# ---------- extractors ----------
def _sum_shap(items):
    """Sum mean_shap from strengths/weaknesses arrays."""
    if not items:
        return 0.0
    return float(sum(x.get("mean_shap", 0.0) for x in items))

def _weapon_ratio(player_json, group, weapon):
    """
    group: 'weapon_usage_ratio' or 'weapon_damage_ratio'
    """
    return float(
        player_json.get("weapon_profile", {})
                  .get(group, {})
                  .get(weapon, 0.0)
    )

def _economy(player_json, key):
    return float(player_json.get("economy_profile", {}).get(key, 0.0))


# ---------- surrogate model ----------
def adjusted_win_probability_player(player_json):
    """
    Returns adjusted win probability for a player_json.
    Uses win_probability.mean as baseline and adjusts it with heuristic signals.
    """
    wp = player_json.get("win_probability", {})
    base_mean = float(wp.get("mean", 0.5))

    # Work in logit space (better additive behavior)
    z = logit(base_mean)

    # ---- strengths & weaknesses ----
    strengths = player_json.get("series_strengths", [])
    weaknesses = player_json.get("series_weaknesses", [])

    shap_strength = _sum_shap(strengths)
    shap_weak = _sum_shap(weaknesses)  # likely negative already

    # give SHAP totals moderate influence
    z += 0.60 * shap_strength
    z += 0.80 * shap_weak

    # ---- stability / consistency ----
    stability = float(wp.get("stability", 0.5))  # 0..1
    # higher stability => more confidence => slightly higher effective p
    z += 0.25 * (stability - 0.5)

    consistency = float(player_json.get("payer_consistency_score", 0.5))
    z += 0.40 * (consistency - 0.5)

    # ---- economy profile ----
    # player_loadout_ratio + networth_ratio high => stronger contributor
    loadout = _economy(player_json, "player_loadout_ratio")
    networth = _economy(player_json, "player_networth_ratio")
    money_left = _economy(player_json, "money_left_ratio")

    z += 0.90 * (loadout - 0.20)
    z += 0.70 * (networth - 0.20)

    # too much money left could mean under-investing (context dependent)
    z -= 0.35 * (money_left - 0.35)

    # ---- weapon profile ----
    # reward rifle/ability usage, penalize over-sniper dependence slightly
    rifle_use = _weapon_ratio(player_json, "weapon_usage_ratio", "rifle")
    ability_use = _weapon_ratio(player_json, "weapon_usage_ratio", "ability")
    sniper_use = _weapon_ratio(player_json, "weapon_usage_ratio", "sniper")

    z += 0.55 * (rifle_use - 0.45)
    z += 0.25 * (ability_use - 0.20)
    z -= 0.25 * (sniper_use - 0.12)

    # ---- playstyle categorical ----
    playstyle = player_json.get("playstyle", "unknown")
    playstyle_bias = {
        "aim_heavy": 0.08,
        "utility_heavy": 0.05,
        "support": 0.03,
        "unknown": 0.0
    }.get(playstyle, 0.0)
    z += playstyle_bias

    p = sigmoid(z)
    return clamp(p)


# ---------- update helpers ----------
def set_weapon_usage_ratio(player_json, weapon, value):
    out = copy.deepcopy(player_json)
    out.setdefault("weapon_profile", {}).setdefault("weapon_usage_ratio", {})[weapon] = float(value)
    return out

def add_weapon_usage_ratio(player_json, weapon, delta):
    out = copy.deepcopy(player_json)
    cur = _weapon_ratio(out, "weapon_usage_ratio", weapon)
    out.setdefault("weapon_profile", {}).setdefault("weapon_usage_ratio", {})[weapon] = float(cur + delta)
    return out

def set_economy_ratio(player_json, key, value):
    out = copy.deepcopy(player_json)
    out.setdefault("economy_profile", {})[key] = float(value)
    return out

def set_consistency(player_json, value):
    out = copy.deepcopy(player_json)
    out["payer_consistency_score"] = float(value)
    return out

def set_strength_shap(player_json, feature, mean_shap):
    """
    Update existing feature in series_strengths, else append.
    """
    out = copy.deepcopy(player_json)
    strengths = out.setdefault("series_strengths", [])
    for item in strengths:
        if item.get("feature") == feature:
            item["mean_shap"] = float(mean_shap)
            return out
    strengths.append({"feature": feature, "mean_shap": float(mean_shap), "direction": "positive"})
    return out

def set_weakness_shap(player_json, feature, mean_shap):
    out = copy.deepcopy(player_json)
    weaknesses = out.setdefault("series_weaknesses", [])
    for item in weaknesses:
        if item.get("feature") == feature:
            item["mean_shap"] = float(mean_shap)
            return out
    weaknesses.append({"feature": feature, "mean_shap": float(mean_shap), "direction": "negative"})
    return out


In [24]:
import copy

def update_player_json(player_json, updates: dict):
    """
    Supported update keys:

    Weapon ratios:
      weapon_usage_ratio.<weapon>              (shorthand)
      weapon_damage_ratio.<weapon>             (shorthand)
      weapon_profile.weapon_usage_ratio.<weapon>
      weapon_profile.weapon_damage_ratio.<weapon>

    Economy:
      economy_profile.<key>

    Consistency:
      payer_consistency_score

    Strength/weakness shap:
      strength.<feature>
      weakness.<feature>

    Other:
      playstyle
    """
    out = copy.deepcopy(player_json)

    for k, v in updates.items():

        # --- weapon usage ratio ---
        if k.startswith("weapon_usage_ratio."):
            weapon = k.split(".", 1)[1]
            out.setdefault("weapon_profile", {}).setdefault("weapon_usage_ratio", {})[weapon] = float(v)

        elif k.startswith("weapon_profile.weapon_usage_ratio."):
            weapon = k.split(".", 2)[2]
            out.setdefault("weapon_profile", {}).setdefault("weapon_usage_ratio", {})[weapon] = float(v)

        # --- weapon damage ratio ---
        elif k.startswith("weapon_damage_ratio."):
            weapon = k.split(".", 1)[1]
            out.setdefault("weapon_profile", {}).setdefault("weapon_damage_ratio", {})[weapon] = float(v)

        elif k.startswith("weapon_profile.weapon_damage_ratio."):
            weapon = k.split(".", 2)[2]
            out.setdefault("weapon_profile", {}).setdefault("weapon_damage_ratio", {})[weapon] = float(v)

        # --- economy ---
        elif k.startswith("economy_profile."):
            key = k.split(".", 1)[1]
            out.setdefault("economy_profile", {})[key] = float(v)

        # --- consistency ---
        elif k == "payer_consistency_score":
            out["payer_consistency_score"] = float(v)

        # --- playstyle ---
        elif k == "playstyle":
            out["playstyle"] = v

        # --- strengths/weaknesses ---
        elif k.startswith("strength."):
            feat = k.split(".", 1)[1]
            strengths = out.setdefault("series_strengths", [])
            found = False
            for item in strengths:
                if item.get("feature") == feat:
                    item["mean_shap"] = float(v)
                    found = True
                    break
            if not found:
                strengths.append({"feature": feat, "mean_shap": float(v), "direction": "positive"})

        elif k.startswith("weakness."):
            feat = k.split(".", 1)[1]
            weaknesses = out.setdefault("series_weaknesses", [])
            found = False
            for item in weaknesses:
                if item.get("feature") == feat:
                    item["mean_shap"] = float(v)
                    found = True
                    break
            if not found:
                weaknesses.append({"feature": feat, "mean_shap": float(v), "direction": "negative"})

        else:
            raise KeyError(f"Unknown update key: {k}")

    return out


# 5) Monte Carlo for player what-if simulation

def monte_carlo_player(player_json, n=5000, seed=42):
    rng = np.random.default_rng(seed)
    probs = []

    for _ in range(n):
        sim = copy.deepcopy(player_json)

        # sample changes
        rifle = _weapon_ratio(sim, "weapon_usage_ratio", "rifle")
        networth = _economy(sim, "player_networth_ratio")
        consistency = float(sim.get("payer_consistency_score", 0.5))

        # random shocks (tune stddevs)
        rifle2 = clamp(rifle + rng.normal(0, 0.03), 0, 1)
        networth2 = clamp(networth + rng.normal(0, 0.02), 0, 1)
        consistency2 = clamp(consistency + rng.normal(0, 0.05), 0, 1)

        sim = set_weapon_usage_ratio(sim, "rifle", rifle2)
        sim = set_economy_ratio(sim, "player_networth_ratio", networth2)
        sim = set_consistency(sim, consistency2)

        probs.append(adjusted_win_probability_player(sim))

    probs = np.array(probs)
    return {
        "mean": float(probs.mean()),
        "p05": float(np.quantile(probs, 0.05)),
        "median": float(np.quantile(probs, 0.50)),
        "p95": float(np.quantile(probs, 0.95)),
        "min": float(probs.min()),
        "max": float(probs.max()),
        "std": float(probs.std()),
    }



In [ ]:

# 1) single point estimate (deterministic)
baseline_p = adjusted_win_probability_player(playerA_json)

scenario_json = update_player_json(playerA_json, {
    "weapon_usage_ratio.rifle": 0.62,
    "weapon_usage_ratio.smg": 0.62,
    "weapon_usage_ratio.shotgun": 0.62,
    "weapon_usage_ratio.pistol": 0.62,
    "weapon_usage_ratio.ability": 0.62,
    
    "weapon_damage_ratio.rifle": 0.70,
    "weapon_damage_ratio.smg": 0.70,
    "weapon_damage_ratio.pistol": 0.70,
    "weapon_damage_ratio.shotgun": 0.70,
    "weapon_damage_ratio.ability": 0.70,

    "economy_profile.player_loadout_ratio": 0.23,
    "economy_profile.player_networth_ratio": 0.23,
    "economy_profile.money_left_ratio": 0.23,
    "payer_consistency_score": 0.78, 
})

scenario_p = adjusted_win_probability_player(scenario_json)

print("Point estimate:")
print("baseline_p:", baseline_p)
print("scenario_p :", scenario_p)
print("delta      :", scenario_p - baseline_p)


# 2) Monte Carlo distribution estimate
baseline_mc = monte_carlo_player(playerA_json, n=10000, seed=42)
scenario_mc = monte_carlo_player(scenario_json, n=10000, seed=42)

print("\nMonte Carlo:")
print("baseline_mc:", baseline_mc)
print("scenario_mc:", scenario_mc)
print("MC mean delta:", scenario_mc["mean"] - baseline_mc["mean"])
print("MC median delta:", scenario_mc["median"] - baseline_mc["median"])


Point estimate:
baseline_p: 0.19969731379030595
scenario_p : 0.2176534086227441
delta      : 0.017956094832438152

Monte Carlo:
baseline_mc: {'mean': 0.19979221229552807, 'p05': 0.19201588016630117, 'median': 0.1997621965318823, 'p95': 0.2078160392108878, 'min': 0.1827544354290314, 'max': 0.21988184965720722, 'std': 0.004777820230416962}
scenario_mc: {'mean': 0.21775178588084196, 'p05': 0.20946205972011092, 'median': 0.21772253824702584, 'p95': 0.22629572291413805, 'min': 0.19956697795752387, 'max': 0.23911061413961732, 'std': 0.005090304943391569}
MC mean delta: 0.017959573585313887
MC median delta: 0.017960341715143546


In [27]:
import numpy as np
import copy
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def player_vs_player_winprob(playerA_json, playerB_json, temperature=1.0):
    """
    Returns win probability for A vs B.
    temperature > 1 => softer probabilities (less confident)
    temperature < 1 => sharper probabilities (more confident)
    """
    pA = adjusted_win_probability_player(playerA_json)
    pB = adjusted_win_probability_player(playerB_json)

    # convert to strength in logit space
    eps = 1e-9
    pA = min(max(pA, eps), 1-eps)
    pB = min(max(pB, eps), 1-eps)

    sA = math.log(pA / (1 - pA))
    sB = math.log(pB / (1 - pB))

    # relative probability
    pA_win = sigmoid((sA - sB) / temperature)
    pB_win = 1 - pA_win

    return {
        "pA_win": float(pA_win),
        "pB_win": float(pB_win),
        "pA_strength": float(pA),
        "pB_strength": float(pB),
    }

def monte_carlo_player_vs_player(playerA_json, playerB_json, n=10000, seed=42, temperature=1.0):
    rng = np.random.default_rng(seed)

    winsA = 0
    probs = []

    for _ in range(n):
        # sample noisy versions of A and B
        A_sim = copy.deepcopy(playerA_json)
        B_sim = copy.deepcopy(playerB_json)

        # --- noise for A ---
        A_sim = set_consistency(A_sim, np.clip(A_sim.get("payer_consistency_score", 0.5) + rng.normal(0, 0.05), 0, 1))
        A_sim = set_weapon_usage_ratio(A_sim, "rifle",
                                       np.clip(A_sim.get("weapon_profile", {}).get("weapon_usage_ratio", {}).get("rifle", 0.45)
                                               + rng.normal(0, 0.03), 0, 1))

        # --- noise for B ---
        B_sim = set_consistency(B_sim, np.clip(B_sim.get("payer_consistency_score", 0.5) + rng.normal(0, 0.05), 0, 1))
        B_sim = set_weapon_usage_ratio(B_sim, "rifle",
                                       np.clip(B_sim.get("weapon_profile", {}).get("weapon_usage_ratio", {}).get("rifle", 0.45)
                                               + rng.normal(0, 0.03), 0, 1))

        # deterministic p(A wins) for this simulation
        out = player_vs_player_winprob(A_sim, B_sim, temperature=temperature)
        pA_win = out["pA_win"]
        probs.append(pA_win)

        # sample actual win
        if rng.random() < pA_win:
            winsA += 1

    probs = np.array(probs)
    return {
        "pA_win_mean": float(probs.mean()),
        "pA_win_median": float(np.quantile(probs, 0.5)),
        "pA_win_p10": float(np.quantile(probs, 0.10)),
        "pA_win_p90": float(np.quantile(probs, 0.90)),
        "pA_win_simulated": float(winsA / n),
        "pB_win_mean": float(1 - probs.mean()),
        "n": n
    }

mc = monte_carlo_player_vs_player(playerA_json, playerB_json, n=20000)
mc

{'pA_win_mean': 0.9504593007937848,
 'pA_win_median': 0.9504597308938969,
 'pA_win_p10': 0.948457536781112,
 'pA_win_p90': 0.9524669420765937,
 'pA_win_simulated': 0.9477,
 'pB_win_mean': 0.04954069920621518,
 'n': 20000}

In [28]:
monte_carlo_player_vs_player(playerB_json, playerA_json, n=20000)

{'pA_win_mean': 0.049547425586048806,
 'pA_win_median': 0.049539840971080676,
 'pA_win_p10': 0.04756639335467283,
 'pA_win_p90': 0.05155141773780635,
 'pA_win_simulated': 0.05015,
 'pB_win_mean': 0.9504525744139511,
 'n': 20000}